In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
%matplotlib inline

In [2]:
class NeuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        self.lr = learningrate
        rng = np.random.default_rng(0)
        self.wih = rng.random((hnodes, inodes)) - self.lr
        self.who = rng.random((onodes, hnodes)) - self.lr
        self.act = lambda x: scipy.special.expit(x)  # sigmoid
        print("wih\n", self.wih)
        print("who\n", self.who)
    
    def train(self, inputs, targets):
        inputs = np.array(inputs, ndmin=2).T  # (3,1)
        print("inputs\n", inputs)
              
        targets = np.array(targets, ndmin=2).T  # (3,1)
        print("targets\n", targets)

        i_h = np.dot(self.wih, inputs)
        print("i_h", i_h)
        
        i_h_act = self.act(i_h)
        print("i_h_act", i_h_act)
        
        h_o = np.dot(self.who, i_h_act)
        print("h_o", h_o)
        
        h_o_act = self.act(h_o)
        print("h_o_act", h_o_act)
        
      
        # backpropagation
        e_h_o = targets - h_o_act
        bp_h_o = np.dot(self.who.T, e_h_o)
        print("e_h_o\n",e_h_o)
            
        #                                          ________matrices_________
        #                             ____________|___________________    __|____
        #                            |                                |  |       |
        # ∆Wjk    =    α    *          Ek   *    Ok   *   (1 - Ok)     *   Oj^T
        self.who += self.lr * np.dot(bp_h_o * h_o_act * (1.0 - h_o_act), i_h_act.T)
        #                                     |_______________________|
        #                                       derivative of sigmoid


        # the same procedure for the previous layer

        e_i_h = targets - i_h_act
        bp_i_h = np.dot(self.who.T, e_i_h)
        print("e_i_h\n",e_i_h)

        self.wih += self.lr * np.dot(bp_i_h * i_h_act * (1.0 - i_h_act), inputs.T)
        
        return self.wih, self.who
    

    def query(self, inputs):
        inputs = np.array(inputs, ndmin=2).T
        print("inputs\n", inputs)

        i_h = np.dot(self.wih, inputs)
        print("i_h", i_h)
        
        i_h_act = self.act(i_h)
        print("i_h_act", i_h_act)
        
        h_o = np.dot(self.who, i_h_act)
        print("h_o", h_o)
        
        h_o_act = self.act(h_o)
        print("h_o_act", h_o_act)
        
        return h_o_act
                   

In [3]:
inodes = 3
hnodes = 3
onodes = 3
lr = .3

n = NeuralNetwork(inodes, hnodes, onodes, lr)

wih
 [[ 0.33696169 -0.03021329 -0.25902648]
 [-0.28347236  0.51327024  0.61275558]
 [ 0.30663578  0.42949656  0.24362499]]
who
 [[ 0.63507242  0.51585355 -0.2972615 ]
 [ 0.55740428 -0.26641442  0.42965545]
 [-0.12434438  0.56317892  0.24146122]]


In [4]:
# number of input, hidden and output nodes
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

# learning rate is 0.3
learning_rate = 0.3

# create instance of neural network
n = NeuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

wih
 [[ 0.33696169 -0.03021329 -0.25902648]
 [-0.28347236  0.51327024  0.61275558]
 [ 0.30663578  0.42949656  0.24362499]]
who
 [[ 0.63507242  0.51585355 -0.2972615 ]
 [ 0.55740428 -0.26641442  0.42965545]
 [-0.12434438  0.56317892  0.24146122]]


In [5]:
n.query([1.0, 0.5, -1.5])

inputs
 [[ 1. ]
 [ 0.5]
 [-1.5]]
i_h [[ 0.71039476]
 [-0.94597061]
 [ 0.15594657]]
i_h_act [[0.67048838]
 [0.27969589]
 [0.53890782]]
h_o [[0.40989425]
 [0.53076275]
 [0.20427271]]
h_o_act [[0.60106252]
 [0.62966099]
 [0.55089134]]


array([[0.60106252],
       [0.62966099],
       [0.55089134]])

In [6]:
n.train([1.0, 0.5, -1.5], [.3, .5, .6])

inputs
 [[ 1. ]
 [ 0.5]
 [-1.5]]
targets
 [[0.3]
 [0.5]
 [0.6]]
i_h [[ 0.71039476]
 [-0.94597061]
 [ 0.15594657]]
i_h_act [[0.67048838]
 [0.27969589]
 [0.53890782]]
h_o [[0.40989425]
 [0.53076275]
 [0.20427271]]
h_o_act [[0.60106252]
 [0.62966099]
 [0.55089134]]
e_h_o
 [[-0.30106252]
 [-0.12966099]
 [ 0.04910866]]
e_i_h
 [[-0.37048838]
 [ 0.22030411]
 [ 0.06109218]]


(array([[ 0.32926716, -0.03406055, -0.24748468],
        [-0.29639066,  0.50681109,  0.63213301],
        [ 0.32324074,  0.43779904,  0.21871755]]),
 array([[ 0.62207016,  0.51042963, -0.30771212],
        [ 0.55303725, -0.26823614,  0.42614543],
        [-0.12207295,  0.56412646,  0.24328689]]))